# Решение задачи предсказания дефолта

In [5]:
# Загрузка данных с kaggle. Нужно загрузить kaggle.json с профиля в Kaggle.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c school-of-quants-hackathon-2025-finals
!unzip school-of-quants-hackathon-2025-finals.zip

mv: cannot stat 'kaggle.json': No such file or directory
school-of-quants-hackathon-2025-finals.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  school-of-quants-hackathon-2025-finals.zip
replace X_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
import joblib

In [8]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
print('X_train.shape =', X_train.shape)
print('y_train.shape =', y_train.shape)
print('X_test.shape  =', X_test.shape)
X_train.head()

X_train.shape = (1827404, 48)
y_train.shape = (1827404, 2)
X_test.shape  = (456852, 48)


,id,credit_number_for_user,days_since_confirmed,maturity_plan,maturity_fact,credit_limit,next_payment_sum,sum_left_to_pay,current_overdue_debt,max_overdue_debt,...,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,credit_type,credit_currency
0,1329062,2,14,9,12,5,2,3,0,2,...,3,3,3,4,3,3,3,4,3,1
1,747472,4,3,1,10,14,2,3,0,2,...,0,0,0,4,3,3,3,4,4,1
2,2217455,12,10,1,8,0,5,1,0,2,...,3,3,3,4,3,3,3,4,4,1
3,1103371,7,17,8,5,6,2,3,0,2,...,3,3,3,4,3,3,3,4,4,1
4,517339,16,10,4,8,14,2,3,0,2,...,3,3,3,4,3,3,3,4,3,1


In [19]:
y_train

,id,flag
0,1329062,False
1,747472,False
2,2217455,False
3,1103371,True
4,517339,False
...,...,...
1827399,159827,False
1827400,1428136,False
1827401,480730,False
1827402,2065245,False


In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1827404 entries, 0 to 1827403
Data columns (total 48 columns):
 #   Column                  Dtype
---  ------                  -----
 0   id                      int64
 1   credit_number_for_user  int64
 2   days_since_confirmed    int64
 3   maturity_plan           int64
 4   maturity_fact           int64
 5   credit_limit            int64
 6   next_payment_sum        int64
 7   sum_left_to_pay         int64
 8   current_overdue_debt    int64
 9   max_overdue_debt        int64
 10  full_credit_cost        int64
 11  overdues_5d             int64
 12  overdues_5d_30d         int64
 13  overdues_30d_60d        int64
 14  overdues_60d_90d        int64
 15  overdues_90d            int64
 16  no_overdues_5d          int64
 17  no_overdues_5d_30d      int64
 18  no_overdues_30d_60d     int64
 19  no_overdues_60d_90d     int64
 20  no_overdues_90d         int64
 21  enc_paym_0              int64
 22  enc_paym_1              int64
 23  enc_pay

## Стратегия предобработки

- Числовые признаки: заполняем пропуски медианой и стандартизируем.
- Категориальные признаки (малое число уникальных значений): OneHotEncoding.
- Признаки, которые уже закодированы индексами интервалов или случайными номерами — обрабатывать как категориальные, если у них мало уникальных значений, иначе как числовые.

Мы автоматически определим типы признаков по числу уникальных значений и dtype.

In [13]:
# Автоматическое разделение признаков на типы и создание трансформера

def build_preprocessor(X, cat_threshold=20):
    """Определяем числовые и категориальные признаки автоматически.
    Если признак имеет dtype 'object' или количество уникальных значений <= cat_threshold — считаем его категориальным.
    """
    nunique = X.nunique(dropna=False)
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    # treat low-cardinality numeric as categorical
    low_card_num = [c for c in numeric_features if nunique[c] <= cat_threshold]
    # features explicitly object -> categorical
    obj_feats = X.select_dtypes(include=['object']).columns.tolist()
    categorical_features = sorted(list(set(low_card_num + obj_feats)))
    # final numeric = numeric_features - categorical_features
    numeric_features = [c for c in numeric_features if c not in categorical_features]

    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ], remainder='drop')

    return preprocessor, numeric_features, categorical_features

# Пример (будет работать только если данные загружены)
try:
    preprocessor, num_feats, cat_feats = build_preprocessor(X_train)
    print('Numeric features (auto):', num_feats)
    print('Categorical features (auto):', cat_feats)
except NameError:
    print('Данные не загружены — пропустим автоматическую детекцию.')

Numeric features (auto): ['id', 'credit_number_for_user']
Categorical features (auto): ['credit_currency', 'credit_limit', 'credit_type', 'current_overdue_debt', 'days_since_confirmed', 'enc_paym_0', 'enc_paym_1', 'enc_paym_10', 'enc_paym_11', 'enc_paym_12', 'enc_paym_13', 'enc_paym_14', 'enc_paym_15', 'enc_paym_16', 'enc_paym_17', 'enc_paym_18', 'enc_paym_19', 'enc_paym_2', 'enc_paym_20', 'enc_paym_21', 'enc_paym_22', 'enc_paym_23', 'enc_paym_24', 'enc_paym_3', 'enc_paym_4', 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9', 'full_credit_cost', 'maturity_fact', 'maturity_plan', 'max_overdue_debt', 'next_payment_sum', 'no_overdues_30d_60d', 'no_overdues_5d', 'no_overdues_5d_30d', 'no_overdues_60d_90d', 'no_overdues_90d', 'overdues_30d_60d', 'overdues_5d', 'overdues_5d_30d', 'overdues_60d_90d', 'overdues_90d', 'sum_left_to_pay']


In [ ]:
# Обучение модели (RandomForest) с кросс-валидацией
model = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1, class_weight='balanced')

try:
    preprocessor, num_feats, cat_feats = build_preprocessor(X_train)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('clf', model)])

    X = X_train.copy()
    y = y_train.copy()
    print(X.shape)
    # если y_train — DataFrame, берем первый столбец
    if hasattr(y, 'shape') and y.shape[1] > 1:
        y = y.iloc[:, 1]
    else:
        y = y.squeeze()

    min_class_count = y.value_counts().min()
    n_splits = min(5, min_class_count)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    aucs = []
    f1s = []
    fold = 0
    for train_idx, val_idx in skf.split(X, y):
        fold += 1
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        pipeline.fit(X_tr, y_tr)
        y_pred_proba = pipeline.predict_proba(X_val)[:, 1]
        y_pred = pipeline.predict(X_val)
        auc = roc_auc_score(y_val, y_pred_proba)
        f1 = f1_score(y_val, y_pred)
        aucs.append(auc)
        f1s.append(f1)
        print(f'Fold {fold}: ROC AUC = {auc:.4f}, F1 = {f1:.4f}')
    print('\nMean ROC AUC:', np.mean(aucs))
    print('Mean F1:', np.mean(f1s))

    # Обучаем на всех данных
    pipeline.fit(X, y)
    # Сохраняем модель
    joblib.dump(pipeline, 'rf_pipeline.joblib')
    print('\nМодель сохранена в rf_pipeline.joblib')
except NameError:
    print('Данные не загружены — сначала загрузите X_train и y_train.')

(1827404, 48)


In [11]:
# Предсказание на X_test и сохранение submission.csv
try:
    if 'pipeline' not in globals():
        pipeline = joblib.load('rf_pipeline.joblib')
        print('Загружена сохранённая модель rf_pipeline.joblib')
    preds_proba = pipeline.predict_proba(X_test)[:, 1]
    preds = (preds_proba >= 0.5).astype(int)
    submission = pd.DataFrame({'id': X_test.index if X_test.index.name is not None else np.arange(len(X_test)), 'flag': preds})
    # Если в X_test есть id поле, используем его
    if 'id' in X_test.columns:
        submission = pd.DataFrame({'id': X_test['id'], 'flag': preds})
    submission.to_csv('submission.csv', index=False)
    print('submission.csv сохранён. Пример:')
    display(submission.head())
except Exception as e:
    print('Ошибка при предсказании или сохранении:', e)
    print('Убедитесь, что X_test загружен и модель обучена.')

Ошибка при предсказании или сохранении: [Errno 2] No such file or directory: 'rf_pipeline.joblib'
Убедитесь, что X_test загружен и модель обучена.


## Советы по улучшению

- Попробовать LightGBM / XGBoost — они часто дают лучшее качество для табличных данных.
- Провести тщательный отбор признаков и создать новые (например, отношение overdue/limit и т.д.).
- Подобрать оптимальные гиперпараметры (GridSearchCV / RandomizedSearchCV / Optuna).
- Попробовать стохастическую стратифицированную валидацию и таргетный инжиниринг.

Если нужно, могу добавить реализацию LightGBM + настройку гиперпараметров.